<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 3.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Credit Risk Assessment

### Credit Risk

**Credit Risk** can be defined as the probability or chance that someone or their counterparty will fail to honor their previous agreement, resulting in financial loss to banking institutions when the client *defaults* on the aforementioned agreement<sup><a href="https://www.risk-officer.com/Credit_Risk.htm">1</a>,</sup><sup><a href="https://www.investopedia.com/terms/c/creditrisk.asp">2</a></sup>. Usually, this occurs because of the inability of clients to repay their loans to said institutions.

<p align=center>
<img src="img/credit_risk.jpg" width="40%"><br>
<i><sup>Image credits: storyset @ <a href="https://www.freepik.com/author/stories">freepik</a>.</sup></i>
</p>

Although it may be impossible to predict exactly which clients will incur money loss for the company, ***Credit Risk Management*** refers exactly to the evaluation of this probability. That is, trying to identify which clients will default on their agreements. This is especially important for **Credit Risk Management** strategies, as it will allow the companies to mitigate money loss, for example, by increasing the interest rates for clients who represent higher risks or by even denying loans<sup><a href="https://www.risk-officer.com/Credit_Risk.htm">1</a>,</sup><sup><a href="https://www.investopedia.com/terms/c/creditrisk.asp">2</a></sup>.

One of the strategies used by lenders to evaluate risk are the *5 Cs of Credit*. Although companies have different ways of measuring these, they offer some insights about the risk of financial loss. The 5 Cs are: **Character**, as in the client's credit history; **Capital**, as in the amount of money they have; **Capacity**, or the debt-to-income ratio; **Collateral**, assets that can back or act as security for the loan; and **Conditions**, as in that is the purpose, amount and rates of the loan<sup><a href="https://www.investopedia.com/terms/f/five-c-credit.asp">3</a></sup>.

However, these are only a few characteristics that can be observed. The companies usually have a lot more information about their clients. Using Machine Learning methods, we can leverage this information, and with it try and predict if they will default or not.

## Goal

The goal of the present analysis is to predict if a bank client will default on their financial agreements with the lending institution or not. This evaluation has to happen before the client takes a loan or a credit card. This prediction needs to minimize false positives — to not negatively impact the clients — but also to prevent money loss on the company's side.

## Initial hypotheses 

There are a few initial hypotheses that we can think of.

* Clients with previously recorded bankruptcies will be more likely to default.

* Clients with previously recorded defaults will be more likely to default again.

* Clients with lower credit scores will also be more likely to default.

On the course of this analysis, we will see how these characteristics will impact on the outcome observed.

## About the dataset

In this notebook, we will use a dataset from a Data Science project that was part of a competition held by [Nubank](https://blog.nubank.com.br/nubank-o-que-e/), a digital financial platform from Brazil. The dataset presents several pieces of information about Nubank's clients and whether they defaulted on their financial obligations or not.

In the dataset, we find several columns:

* `ids` = These are the clients' IDs. This column is anonymised. 
* `target_default` = Whether the client defaulted or not. This will be our target variable.
* `score_1` = Credit Score. This column is anonymised.
* `score_2` = Another type of Credit Score. This is also anonymised.
* `score_3` = A third type of Credit Score. Contains the actual numbers.
* `score_4` = A fourth type of Credit Score. Contains the actual numbers.
* `score_5` = A fifth type of Credit Score. Contains the actual numbers.
* `score_6` = A sixth type of Credit Score. Contains the actual numbers.
* `risk_rate` = Unclear. Could be "Interest Rate Risk", which is defined as "the danger that a bank may incur loss or lose money in granting loans (...)".<sup><a href="https://www.sciencedirect.com/topics/economics-econometrics-and-finance/interest-rate-risk">4</a></sup>
* `last_amount_borrowed` = Last amount borrowed by the client.
* `last_borrowed_in_months` = How many months since the last loan.
* `credit_limit` = Limit of credit.
* `reason` = Unclear. This could be the reason for the loan. This is also anonymised.
* `income` = The client's income, probably annual income. 
* `facebook_profile` = If the client has a facebook profile.
* `state` = Geographical state. This is also anonymised.
* `zip` = Zip code. This is also anonymised.
* `channel` = Unclear. This is also anonymised.
* `job_name` = The client's job title. This is also anonymised.
* `real_state` = Unclear. This is also anonymised.
* `ok_since` = Unclear. This is probably a time variable.
* `n_bankruptcies` = Number of previous bankruptcies.
* `n_defaulted_loans` = Number of previous defaulted loans.
* `n_accounts` = Number of accounts.
* `n_issues` = Number of issues.
* `application_time_applied` = The time the application was made.
* `application_time_in_funnel` = How long the application was in "funnel".
* `email` = The client's e-mail provider.
* `external_data_provider_credit_checks_last_2_year` = External data. Credit checks in the last 2 years.
* `external_data_provider_credit_checks_last_month` = External data. Credit checks in the last month.
* `external_data_provider_credit_checks_last_year` = External data. Credit checks in the last year.
* `external_data_provider_email_seen_before` = External data. Unclear. Probably how many times the e-mail was seen previously.
* `external_data_provider_first_name` = External data. First name, unsure if from provider or client. We'll check this information based on the number of unique values.
* `external_data_provider_fraud_score` = External data. Fraud score.
* `lat_lon` = Latitude and Longitude of the client.
* `marketing_channel` = Marketing channel through which the client decided on a loan.
* `profile_phone_number` = Client's phone number. Looks anonymised.
* `reported_income` = Client's reported income.
* `shipping_state` = Country state for shipping to the client.
* `shipping_zip_code` = Zip code for shipping to the client. Looks anonymised.
* `profile_tags` = Dictionary of tags for each client. Unclear to what it means.
* `user_agent` = Information about which platform was used by the client (navigator, operating system, et cetera).
* `target_fraud` = Fraud information for another analysis in the same dataset.

## Importing data

Let's start our analysis by importing our dependencies, setting some parameters and reading our dataset. We will also print the first few entries of the data.

In [100]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tools as mytools
from geopy.geocoders import Nominatim

# Options
pd.set_option('display.max_columns', None)

# The Moon also rises
np.random.seed(6327)

# Defining plot parameters
# plt.style.use('dark_background')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.stretch'] = 'normal'
plt.rcParams['font.style'] = 'normal'
plt.rcParams['font.variant'] = 'normal'

# Reading dataframe
df = pd.read_csv("data/acquisition_train.csv")

In [101]:
# Checking size and first entries
print(df.shape)
df.head(3)

(45000, 43)


,ids,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,reason,income,facebook_profile,state,zip,channel,job_name,real_state,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_applied,application_time_in_funnel,email,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_first_name,external_data_provider_fraud_score,lat_lon,marketing_channel,profile_phone_number,reported_income,shipping_state,shipping_zip_code,profile_tags,user_agent,target_fraud
0,343b7e7b-2cf8-e508-b8fd-0a0285af30aa,False,1Rk8w4Ucd5yR3KcqZzLdow==,IOVu8au3ISbo6+zmfnYwMg==,350.0,101.800832,0.259555,108.427273,0.40,25033.92,36.0,0.0,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,65014.12,True,sjJbkqJS7cXalHLBFA+EOQ==,Ernn+uVXCMq/6ARrBCcd+A==,NCqL3QBx0pscDnx3ixKwXg==,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,N5/CE7lSkAfB04hVFFwllw==,NaN,0.0,0.0,18.0,18.0,07:52:34,444,outlook.com,NaN,2,0.0,51.0,leidelaura,645,"(-29.151545708122246, -51.1386461804385)",Invite-email,514-9840782,57849.0,BR-MT,17528,"{'tags': ['n19', 'n8']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,NaN
1,bc2c7502-bbad-0f8c-39c3-94e881967124,False,DGCQep2AE5QRkNCshIAlFQ==,SaamrHMo23l/3TwXOWgVzw==,370.0,97.062615,0.942655,92.002546,0.24,NaN,NaN,39726.0,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,100018.91,False,xsd3ZdsI3356I3xMxZeiqQ==,rlWIXTBO+VOa34+SpGyhlQ==,NCqL3QBx0pscDnx3ixKwXg==,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,N5/CE7lSkAfB04hVFFwllw==,50.0,0.0,0.0,14.0,14.0,02:34:29,346,gmail.com,0.0,1,0.0,17.0,diocezio,243,"(-19.687710705798963, -47.94151536525154)",Radio-commercial,251-3659293,4902.0,BR-RS,40933,"{'tags': ['n6', 'n7', 'nim']}",Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-...,NaN
2,669630dd-2e6a-0396-84bf-455e5009c922,True,DGCQep2AE5QRkNCshIAlFQ==,Fv28Bz0YRTVAT5kl1bAV6g==,360.0,100.027073,0.351918,112.892453,0.29,7207.92,36.0,NaN,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,65023.65,NaN,Ygq6MsM98oC8yceExr69Ig==,PjTIDfJsK0DKL9fO7vuW2g==,NCqL3QBx0pscDnx3ixKwXg==,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,n+xK9CfX0bCn77lClTWviw==,NaN,0.0,0.0,10.0,NaN,00:60:02,6,gmail.com,NaN,2,NaN,9.0,veralucia,65,"(-28.748023890412284, -51.867279334353995)",Waiting-list,230-6097993,163679.0,BR-RR,50985,"{'tags': ['n0', 'n17', 'nim', 'da']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,NaN


In [102]:
df.describe()

,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income,shipping_zip_code
count,44438.000000,45000.000000,45000.000000,45000.000000,44438.000000,15044.000000,15044.000000,31200.000000,4.443800e+04,18455.000000,44303.000000,44426.000000,44438.000000,33456.000000,45000.000000,22372.0,45000.000000,29876.000000,42767.000000,45000.000000,45000.00,45000.000000
mean,347.983258,100.006797,0.500037,99.925228,0.296717,13782.341991,41.763893,34381.951154,7.160739e+04,35.069737,0.076270,0.004569,10.696701,11.068119,247.537000,0.0,1.501067,0.503381,12.534781,500.399800,inf,33923.514844
std,110.429165,3.178080,0.288326,10.016549,0.101933,8077.571933,10.252705,36586.982218,5.185897e+04,21.663194,0.274052,0.079400,4.592463,4.595501,146.118398,0.0,1.114941,0.499997,126.530487,287.968873,NaN,12692.406382
min,0.000000,86.191572,0.000035,60.663039,0.000000,1005.180000,36.000000,0.000000,4.821180e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-999.000000,0.000000,403.00,12001.000000
25%,270.000000,97.867036,0.252150,93.198353,0.220000,7519.502500,36.000000,10000.000000,4.501078e+04,17.000000,0.000000,0.000000,7.000000,8.000000,120.000000,0.0,1.000000,0.000000,11.000000,252.000000,50948.25,22937.000000
50%,340.000000,100.018900,0.500753,99.974796,0.290000,12023.465000,36.000000,25647.000000,6.126676e+04,32.000000,0.000000,0.000000,10.000000,10.000000,248.000000,0.0,2.000000,1.000000,27.000000,503.000000,101779.00,33896.000000
75%,420.000000,102.141797,0.748816,106.640275,0.360000,19220.270000,36.000000,47086.750000,8.601923e+04,50.000000,0.000000,0.000000,13.000000,14.000000,375.000000,0.0,2.000000,1.000000,43.000000,747.000000,151337.25,44910.000000
max,990.000000,113.978234,0.999973,142.192400,0.900000,35059.600000,60.000000,448269.000000,5.000028e+06,141.000000,5.000000,5.000000,49.000000,49.000000,500.000000,0.0,3.000000,1.000000,59.000000,1000.000000,inf,56000.000000


### The first look

As we can see, some of our columns are censored or anonymised. These include columns `score_1` and `score_2` and some clients' personal information such as `reason` and those related to their addresses and occupations.

However, there are coordinates which have been provided for each client (`lat_lon` variable). Whether these relate to their living location or the place where they contacted the bank for their loan* is unclear, although the latter may be more likely. Whatever the case, we can extract location information from this. However, we already have a `shipping_state` variable with location information.

<sup><sub>*_**Observation**: We do not know if the service acquired by the clients here was an actual loan or credit card service. Either way, these represent the bank "lending" money to the client. Thus, from now on in this notebook, we will refer to this contract made between both parties as a **loan** for the client._</sub></sup>

We have two variables with the client's income. One, `income`, is probably derived from the money received in the account and the other, `reported_income`, is likely the income the client has reported when creating the account. These values do not match but are likely correlated. If they are correlated, this could be redundant to our model. Instead, we can code a column showing if the reported income matches the actual account data. These values will hardly match 100%, so we can establish a deviation threshold for this.

Although `target_fraud` is not our target column for this analysis, it might provide some information for our model if the entry was subject to fraud before or not.

In addition, the variable `external_data_provider_email_seen_before` has -999 as its lowest value, and will be replaced by NaNs.

In [103]:
# Fixing -999s
df.external_data_provider_email_seen_before = df.external_data_provider_email_seen_before.replace(-999, np.nan)
df.external_data_provider_email_seen_before.describe()

count    42121.000000
mean        28.048456
std         17.939836
min         -1.000000
25%         12.000000
50%         28.000000
75%         44.000000
max         59.000000
Name: external_data_provider_email_seen_before, dtype: float64

We can see that the variable still has -1 data points. If this is count data from the e-mail, it can't have negative values. We will change these -1s to NaN as well.

In [104]:
# Fixing -999s
df.external_data_provider_email_seen_before = df.external_data_provider_email_seen_before.replace(-1, np.nan)
df.external_data_provider_email_seen_before.describe()

count    41474.000000
mean        28.501615
std         17.705628
min          0.000000
25%         13.000000
50%         28.000000
75%         44.000000
max         59.000000
Name: external_data_provider_email_seen_before, dtype: float64

Now it looks "correct".


## Dealing with categorical data

Let us check our column types and empty values.


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 43 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   ids                                               45000 non-null  object 
 1   target_default                                    41741 non-null  object 
 2   score_1                                           44438 non-null  object 
 3   score_2                                           44438 non-null  object 
 4   score_3                                           44438 non-null  float64
 5   score_4                                           45000 non-null  float64
 6   score_5                                           45000 non-null  float64
 7   score_6                                           45000 non-null  float64
 8   risk_rate                                         44438 non-null  float64
 9   last_amount_borro

We have many columns with empty values, which will investigate further.

In [106]:
# Before proceeding, get column types
num_cols, bcat_cols, mcat_cols = mytools.classifyColumns(df)

<sup><sub>Observation: The `mytools` module is a collection of functions I wrote or gathered along the way while building my projects. They can be checked in the `tools.py` file in this same folder.</sub></sup>

The `target_default` column has some empty values. Let's see our value counts for it.

In [107]:
# Checking target column null values percentagem
df.target_default.isna().value_counts()/df.shape[0]*100

False    92.757778
True      7.242222
Name: target_default, dtype: float64

To aid in our following analysis, we will go ahead and drop these rows.

In [108]:
# Creating another df 
df_clean = df.copy()

# Since there are only a few empty values in our target variable, we drop the rows without the information
df_clean.dropna(subset=['target_default'], inplace=True)

Let's check our unique values per column in our categorical variables.

In [109]:
# Binary columns
mytools.uniqueValuesPerColumn(df_clean[bcat_cols])

,Variable,Unique values,Categories,NaNs
0,channel,NCqL3QBx0pscDnx3ixKwXg==,1,0


The column `channel` seem to contain only one value. We'll also drop this column, as it is non-informative.

In [110]:
# Multiple categories columns
mytools.uniqueValuesPerColumn(df_clean[mcat_cols])

,Variable,Unique values,Categories,NaNs
0,ids,"343b7e7b-2cf8-e508-b8fd-0a0285af30aa, bc2c7502...",41741,0
1,target_default,"False, True",2,0
2,score_1,"1Rk8w4Ucd5yR3KcqZzLdow==, DGCQep2AE5QRkNCshIAl...",7,0
3,score_2,"IOVu8au3ISbo6+zmfnYwMg==, SaamrHMo23l/3TwXOWgV...",35,0
4,reason,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,14261,4
5,facebook_profile,"True, False, nan",3,4153
6,state,"sjJbkqJS7cXalHLBFA+EOQ==, xsd3ZdsI3356I3xMxZei...",50,0
7,zip,"Ernn+uVXCMq/6ARrBCcd+A==, rlWIXTBO+VOa34+SpGyh...",823,0
8,job_name,mLVIVxoGY7TUDJ1FyFoSIZi1SFcaBmO01AydRchaEiGYtU...,30544,2617
9,real_state,"N5/CE7lSkAfB04hVFFwllw==, n+xK9CfX0bCn77lClTWv...",5,0


### Removing non-informative data

The `ids` column contains _only_ unique values, and thus will not be informative to our model later on. The `profile_phone_number` behaves in the same way. The `target_fraud` column values are unclear and represent way too many empty values, so we will drop these columns.

The variables `job_name` and `external_data_provider_first_name` have too many unique values and seem non-informative and will be removed.

`state` and `zip` do not look like they correspond to actual addresses and seem non-informative as well. They will also be removed.

The `reason` column seems to be why the loan was taken, and only has 4 missing values. However, over 14,000 unique values seem non-informative as well. If this column was not censored, we could probably look at the messy data and coalesce common reasons for loans, which could help the model. Thus, we will also drop this column.

We could extract location information from the `lat_lon` variable but would lose more observations in the dataset due to the `1262` missing values it has. Besides, with the amount of entries in our dataset, this would take too long.

Column `application_time_applied` could be _"binned"_ into "hour of the day" to provide more concise information. However, this column seems to contain invalid time data. As we are unclear about the contents, we will drop this column too.

In [111]:
# Checking time column
df_clean.application_time_applied

0        07:52:34
1        02:34:29
2        00:60:02
3        11:20:49
4        13:39:03
           ...   
44995    13:27:39
44996    22:50:49
44997    09:60:49
44998    02:18:51
44999    15:01:57
Name: application_time_applied, Length: 41741, dtype: object

In [112]:
# Cleaning
df_clean.drop(['ids', 'profile_phone_number', 'channel', 'target_fraud',
               'job_name', 'external_data_provider_first_name', 'state',
               'zip', 'reason', 'lat_lon', 'application_time_applied'], axis=1, inplace=True)

In [113]:
# Before proceeding, get column types again
num_cols, bcat_cols, mcat_cols = mytools.classifyColumns(df_clean)

Here is the remaining columns we have to deal with:

In [114]:
# Checking columns again
mytools.uniqueValuesPerColumn(df_clean[mcat_cols])

,Variable,Unique values,Categories,NaNs
0,score_1,"1Rk8w4Ucd5yR3KcqZzLdow==, DGCQep2AE5QRkNCshIAl...",7,0
1,score_2,"IOVu8au3ISbo6+zmfnYwMg==, SaamrHMo23l/3TwXOWgV...",35,0
2,facebook_profile,"True, False, nan",3,4153
3,real_state,"N5/CE7lSkAfB04hVFFwllw==, n+xK9CfX0bCn77lClTWv...",5,0
4,email,"outlook.com, gmail.com, spgov.com, hotmail.com...",6,0
5,marketing_channel,"Invite-email, Radio-commercial, Waiting-list, ...",10,3308
6,shipping_state,"BR-MT, BR-RS, BR-RR, BR-RN, BR-SP, BR-AC, BR-M...",25,0
7,profile_tags,"{'tags': ['n19', 'n8']}, {'tags': ['n6', 'n7',...",24458,0
8,user_agent,Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,298,656


### Inputing missing categorical data

The columns `facebook_profile` and `marketing_channel` look like optional columns and thus may not have been informed by the client. It is reasonable to impute the empty values in this scenario with the most commonly found value in each respective column.

<sup><sub>Note: Another reasonable action for the `marketing_channel` column would be to just use a category termed _"none"_ or _"other"_, as it could mean that these clients reached the negotiation channel without  being targeted by any of the previously listed marketing strategies.</sub></sup>

In [115]:
# Value counts for the facebook_profile column
df_clean.facebook_profile.value_counts()

False    23653
True     13935
Name: facebook_profile, dtype: int64

In [116]:
# Value counts for the marketing_channel column
df_clean.marketing_channel.value_counts()

Website             10163
Waiting-list         6491
Invite-web           3247
Radio-commercial     3145
Facebook             3109
LinkedIn             3083
Instagram            3077
Twitter              3070
Invite-email         3048
Name: marketing_channel, dtype: int64

In [117]:
from sklearn.impute import SimpleImputer

# Replacing empty values in categorical variables
vars = ['facebook_profile', 'marketing_channel']
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(df_clean.loc[:,vars])
df_clean.loc[:,vars] = imputer.transform(df_clean.loc[:,vars])

C:\Users\Carlos\AppData\Local\Temp/ipykernel_6420/58105084.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_clean.loc[:,vars] = imputer.transform(df_clean.loc[:,vars])


In [118]:
# Checking columns again
mytools.uniqueValuesPerColumn(df_clean[mcat_cols])

,Variable,Unique values,Categories,NaNs
0,score_1,"1Rk8w4Ucd5yR3KcqZzLdow==, DGCQep2AE5QRkNCshIAl...",7,0
1,score_2,"IOVu8au3ISbo6+zmfnYwMg==, SaamrHMo23l/3TwXOWgV...",35,0
2,facebook_profile,"True, False",2,0
3,real_state,"N5/CE7lSkAfB04hVFFwllw==, n+xK9CfX0bCn77lClTWv...",5,0
4,email,"outlook.com, gmail.com, spgov.com, hotmail.com...",6,0
5,marketing_channel,"Invite-email, Radio-commercial, Waiting-list, ...",9,0
6,shipping_state,"BR-MT, BR-RS, BR-RR, BR-RN, BR-SP, BR-AC, BR-M...",25,0
7,profile_tags,"{'tags': ['n19', 'n8']}, {'tags': ['n6', 'n7',...",24458,0
8,user_agent,Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,298,656


### Feature engineering

The `profile_tags` can be converted to individual columns with each tag. They do not seem to mean anything at first sight but could mean something to the bank.

The `user_agent` seems to have information on which device was used to contact the bank.

In [119]:
## Converting tags to individual columns

# First, create another df and evaluate the column to a dict
tags_df = pd.json_normalize(df_clean['profile_tags'].map(eval))
tags_df.index = df_clean.index

# Create another column to preserve the order of rows
tags_df['id'] = tags_df.index

# Check df
tags_df.head()

,tags,id
0,"[n19, n8]",0
1,"[n6, n7, nim]",1
2,"[n0, n17, nim, da]",2
3,[n4],3
4,"[pro+aty, n19, da, b19]",4


In [120]:
# Explode tags into individual values for each row
tags_df = tags_df.explode('tags')

# Counting each occurrence for each row
tags_count = pd.crosstab(tags_df['id'], tags_df['tags'])
tags_count.head()

tags,a15,aty,b19,c1,c55,da,dfa,n0,n1,n10,n11,n12,n13,n14,n15,n16,n17,n18,n19,n2,n3,n4,n5,n6,n7,n8,n9,nim,pro+aty
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
2,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [121]:
# Checking if indexes match
(tags_count.index==df_clean.index).all()

True

In [122]:
# Joining two dataframes
df_clean = pd.concat([df_clean, tags_count], axis=1)
df_clean.head()

,target_default,score_1,score_2,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,facebook_profile,real_state,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,email,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,marketing_channel,reported_income,shipping_state,shipping_zip_code,profile_tags,user_agent,a15,aty,b19,c1,c55,da,dfa,n0,n1,n10,n11,n12,n13,n14,n15,n16,n17,n18,n19,n2,n3,n4,n5,n6,n7,n8,n9,nim,pro+aty
0,False,1Rk8w4Ucd5yR3KcqZzLdow==,IOVu8au3ISbo6+zmfnYwMg==,350.0,101.800832,0.259555,108.427273,0.40,25033.92,36.0,0.0,65014.12,True,N5/CE7lSkAfB04hVFFwllw==,NaN,0.0,0.0,18.0,18.0,444,outlook.com,NaN,2,0.0,51.0,645,Invite-email,57849.0,BR-MT,17528,"{'tags': ['n19', 'n8']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,False,DGCQep2AE5QRkNCshIAlFQ==,SaamrHMo23l/3TwXOWgVzw==,370.0,97.062615,0.942655,92.002546,0.24,NaN,NaN,39726.0,100018.91,False,N5/CE7lSkAfB04hVFFwllw==,50.0,0.0,0.0,14.0,14.0,346,gmail.com,0.0,1,0.0,17.0,243,Radio-commercial,4902.0,BR-RS,40933,"{'tags': ['n6', 'n7', 'nim']}",Mozilla/5.0 (Linux; Android 5.0.2; SAMSUNG SM-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
2,True,DGCQep2AE5QRkNCshIAlFQ==,Fv28Bz0YRTVAT5kl1bAV6g==,360.0,100.027073,0.351918,112.892453,0.29,7207.92,36.0,NaN,65023.65,False,n+xK9CfX0bCn77lClTWviw==,NaN,0.0,0.0,10.0,NaN,6,gmail.com,NaN,2,NaN,9.0,65,Waiting-list,163679.0,BR-RR,50985,"{'tags': ['n0', 'n17', 'nim', 'da']}",Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
3,False,1Rk8w4Ucd5yR3KcqZzLdow==,dCm9hFKfdRm7ej3jW+gyxw==,510.0,101.599485,0.987673,94.902491,0.32,NaN,NaN,54591.0,68830.01,False,n+xK9CfX0bCn77lClTWviw==,NaN,1.0,0.0,19.0,19.0,406,spgov.com,NaN,3,NaN,38.0,815,Waiting-list,1086.0,BR-RN,37825,{'tags': ['n4']},Mozilla/5.0 (Linux; Android 6.0; HTC One X10 B...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,False,8k8UDR4Yx0qasAjkGrUZLw==,+CxEO4w7jv3QPI/BQbyqAA==,500.0,98.474289,0.532539,118.126207,0.18,NaN,NaN,NaN,60011.29,True,n+xK9CfX0bCn77lClTWviw==,NaN,0.0,0.0,11.0,NaN,240,gmail.com,0.0,2,1.0,46.0,320,Invite-email,198618.0,BR-MT,52827,"{'tags': ['pro+aty', 'n19', 'da', 'b19']}",Mozilla/5.0 (Linux; Android 7.0; Pixel C Build...,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


Our columns appear to match the `profile_tags` data, therefore we will exclude the old column.

In [123]:
# Cleaning
df_clean.drop(['profile_tags'], axis=1, inplace=True)

Now it is time to deal with the `user_agent` data. Let us look at the entries:

In [124]:
with pd.option_context("display.max_rows", 1000, "display.max_colwidth", 200):
    print(pd.DataFrame(df.user_agent.value_counts().head(16)))

                                                                                                                                                         user_agent
Mozilla/5.0 (Linux; Android 5.0.2; LG-V410/V41020c Build/LRX22G) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/34.0.1847.118 Safari/537.362         195
Mozilla/5.0 (Linux; Android 7.0; Pixel C Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.98 Safari/537.369                190
Mozilla/5.0 (Linux; Android 7.0; Pixel C Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.98 Safari/537.363                189
Mozilla/5.0 (Linux; Android 7.0; SM-T827R4 Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.116 Safari/537.368                             189
Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Build/32.2.A.0.253; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/52.0.2743.98 Safari/537.364         187
Mozilla/5.0 (X11

In the beginning, we can find information about the device used to access and the operational system (OS). We can build a regex search to extract this information. However, we will not have the device model for every entry as only Android specifies it. Instead, we will retrieve generic information about the type of system used. I wrote a wrapper function around the functionality of the `python_user_agent` package to do this job.

In [125]:
# Only a few empty values so let's delete them
df_clean.dropna(subset=['user_agent'], inplace=True)

In [126]:
# Retrieving user_agent info
df_clean = mytools.userAgentInfo(df_clean, 'user_agent')

In [127]:
# Checking entries
df_clean.iloc[:,-7:].head()

,ua_browser,ua_os,ua_device,ua_device_brand,ua_is_mobile,ua_is_tablet,ua_is_pc
0,Chrome Mobile WebView,Android,SGP771,Sony,True,True,False
1,Samsung Internet,Android,Samsung SM-T550,Samsung,False,True,False
2,Chrome Mobile WebView,Android,SGP771,Sony,True,True,False
3,Chrome Mobile WebView,Android,HTC One X10,HTC,True,False,False
4,Chrome Mobile WebView,Android,Pixel C,Google,True,True,False


In [128]:
# Before proceeding, get column types again
num_cols, bcat_cols, mcat_cols = mytools.classifyColumns(df_clean)

# Checking columns again
mytools.uniqueValuesPerColumn(df_clean[mcat_cols])

,Variable,Unique values,Categories,NaNs
0,score_1,"1Rk8w4Ucd5yR3KcqZzLdow==, DGCQep2AE5QRkNCshIAl...",7,0
1,score_2,"IOVu8au3ISbo6+zmfnYwMg==, SaamrHMo23l/3TwXOWgV...",35,0
2,real_state,"N5/CE7lSkAfB04hVFFwllw==, n+xK9CfX0bCn77lClTWv...",5,0
3,email,"outlook.com, gmail.com, spgov.com, hotmail.com...",6,0
4,marketing_channel,"Invite-email, Radio-commercial, Waiting-list, ...",9,0
5,shipping_state,"BR-MT, BR-RS, BR-RR, BR-RN, BR-SP, BR-AC, BR-M...",25,0
6,user_agent,Mozilla/5.0 (Linux; Android 6.0.1; SGP771 Buil...,297,0
7,ua_browser,"Chrome Mobile WebView, Samsung Internet, Chrom...",9,0
8,ua_os,"Android, iOS, Ubuntu, Mac OS X, Windows, Chrom...",6,0
9,ua_device,"SGP771, Samsung SM-T550, HTC One X10, Pixel C,...",19,0


In [129]:
# Dropping ua_device_brand as it has too many missing values and the old user_agent column
df_clean.drop(['ua_device_brand', 'user_agent'], axis=1, inplace=True)

In [130]:
# Checking binary columns again
mytools.uniqueValuesPerColumn(df_clean[bcat_cols])

,Variable,Unique values,Categories,NaNs
0,target_default,"False, True",2,0
1,ua_is_mobile,"True, False",2,0
2,ua_is_tablet,"True, False",2,0
3,ua_is_pc,"False, True",2,0


## Dealing with numeric data

Now, we will look at the numeric columns in our dataset.

In [131]:
# Checking first entries.
df_clean[num_cols].iloc[:,0:22].head()

,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income,shipping_zip_code
0,350.0,101.800832,0.259555,108.427273,0.40,25033.92,36.0,0.0,65014.12,NaN,0.0,0.0,18.0,18.0,444,NaN,2,0.0,51.0,645,57849.0,17528
1,370.0,97.062615,0.942655,92.002546,0.24,NaN,NaN,39726.0,100018.91,50.0,0.0,0.0,14.0,14.0,346,0.0,1,0.0,17.0,243,4902.0,40933
2,360.0,100.027073,0.351918,112.892453,0.29,7207.92,36.0,NaN,65023.65,NaN,0.0,0.0,10.0,NaN,6,NaN,2,NaN,9.0,65,163679.0,50985
3,510.0,101.599485,0.987673,94.902491,0.32,NaN,NaN,54591.0,68830.01,NaN,1.0,0.0,19.0,19.0,406,NaN,3,NaN,38.0,815,1086.0,37825
4,500.0,98.474289,0.532539,118.126207,0.18,NaN,NaN,NaN,60011.29,NaN,0.0,0.0,11.0,NaN,240,0.0,2,1.0,46.0,320,198618.0,52827


In [132]:
# Checking numeric columns
mytools.uniqueValuesPerColumn(df_clean[num_cols].iloc[:,0:22])

,Variable,Unique values,Categories,NaNs
0,score_3,"350.0, 370.0, 360.0, 510.0, 500.0, 300.0, 250....",87,0
1,score_4,"101.800831713, 97.0626153138, 100.027072525, 1...",41085,0
2,score_5,"0.259554667831, 0.942654520464, 0.351917595299...",41085,0
3,score_6,"108.427272823, 92.002545526, 112.892452948, 94...",41085,0
4,risk_rate,"0.4, 0.24, 0.29, 0.32, 0.18, 0.44, 0.43, 0.33,...",81,0
5,last_amount_borrowed,"25033.92, nan, 7207.92, 5019.53, 7524.9, 20034...",13277,27174
6,last_borrowed_in_months,"36.0, nan, 60.0",3,27174
7,credit_limit,"0.0, 39726.0, nan, 54591.0, 61055.0, 42666.0, ...",19107,12887
8,income,"65014.12, 100018.91, 65023.65, 68830.01, 60011...",38255,0
9,ok_since,"nan, 50.0, 11.0, 35.0, 16.0, 19.0, 23.0, 43.0,...",100,24080


In [133]:
# Checking summary
df_clean[num_cols].iloc[:,0:22].describe()

,score_3,score_4,score_5,score_6,risk_rate,last_amount_borrowed,last_borrowed_in_months,credit_limit,income,ok_since,n_bankruptcies,n_defaulted_loans,n_accounts,n_issues,application_time_in_funnel,external_data_provider_credit_checks_last_2_year,external_data_provider_credit_checks_last_month,external_data_provider_credit_checks_last_year,external_data_provider_email_seen_before,external_data_provider_fraud_score,reported_income,shipping_zip_code
count,41085.000000,41085.000000,41085.000000,41085.000000,41085.000000,13911.000000,13911.000000,28198.000000,4.108500e+04,17005.000000,40953.000000,41073.000000,41085.000000,30341.000000,41085.000000,20399.0,41085.000000,27281.000000,37867.000000,41085.000000,41085.0,41085.000000
mean,346.491907,100.007088,0.499434,99.919927,0.294565,13320.242985,40.582273,33854.191077,7.109676e+04,35.174772,0.076869,0.004602,10.635414,11.016974,247.814141,0.0,1.504929,0.503317,28.456281,500.379579,inf,33901.092442
std,110.160494,3.181303,0.288094,10.026968,0.101564,7925.847357,9.433118,36133.109320,5.241961e+04,21.623677,0.275223,0.080193,4.580684,4.590187,146.419733,0.0,1.113969,0.499998,17.716520,288.061125,NaN,12696.821357
min,0.000000,86.191572,0.000035,60.663039,0.000000,1005.180000,36.000000,0.000000,4.821180e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,403.0,12001.000000
25%,270.000000,97.864991,0.251607,93.184756,0.220000,7206.055000,36.000000,9999.000000,4.402151e+04,17.000000,0.000000,0.000000,7.000000,8.000000,120.000000,0.0,1.000000,0.000000,13.000000,252.000000,50909.0,22899.000000
50%,340.000000,100.021020,0.500219,99.979525,0.290000,12010.470000,36.000000,25179.500000,6.004454e+04,32.000000,0.000000,0.000000,10.000000,10.000000,248.000000,0.0,2.000000,1.000000,28.000000,502.000000,101626.0,33878.000000
75%,420.000000,102.138975,0.747656,106.630297,0.360000,18029.515000,36.000000,46449.750000,8.503268e+04,50.000000,0.000000,0.000000,13.000000,14.000000,375.000000,0.0,2.000000,1.000000,44.000000,747.000000,151250.0,44911.000000
max,990.000000,113.978234,0.999973,142.192400,0.900000,35059.600000,60.000000,448269.000000,5.000028e+06,141.000000,5.000000,5.000000,49.000000,49.000000,500.000000,0.0,3.000000,1.000000,59.000000,1000.000000,inf,55998.000000


The column `ok_since` has unclear content and many missing values. The column `shipping_zip_code`, which should be a categorical variable, appears here without actual zip codes. The column `external_data_provider_credit_checks_last_2_year` does not seem to contain valid information, as it only contains NaN or zeroes. These columns will be removed.

In [134]:
# Cleaning data
df_clean.drop(['ok_since', 'shipping_zip_code', 'external_data_provider_credit_checks_last_2_year'], axis=1, inplace=True)

### Inputing missing data

Both `last_amount_borrowed` and `last_borrowed_in_months` present a high number of NaN values. It seems reasonable to assume that if the value is empty, that no value was borrowed within the mentioned period, so we will replace these NaNs with zeroes. The same reasoning goes for `credit_limit` (empty value meaning client has no credit), `n_bankruptcies`, `n_defaulted_loans` and `n_issues` (empty value meaning no bankruptcies, defaulted loans or issues).

The column `external_data_provider_credit_checks_last_year` seems to contain information about wether or not credit checks were made in the last year (0 or 1 values). Thus, it seems reasonable to input missing data with zeroes, too.

In [136]:
# Defining columns
fill_with_zeroes = ['last_amount_borrowed', 'last_borrowed_in_months', 'credit_limit', 'n_issues',
                    'n_bankruptcies', 'n_defaulted_loans', 'external_data_provider_credit_checks_last_year']

# Filling NaNs with zeroes
for col in fill_with_zeroes:
    df_clean[col].fillna(value=0, inplace=True)

The only column left to deal with is `external_data_provider_email_seen_before`. Due to the amount of missing values (3218) and the hard-to-understand nature of the way these variables are coded, we will inpute the missing values with the median value as it is a good measure to describe the data.

In [143]:
# Writing column
col = 'external_data_provider_email_seen_before'

# Imputing NaNs with median
df_clean[col] = df_clean[col].fillna(df_clean[col].median())

So if we look at our dataset again:

In [145]:
# Checking our dataset
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41085 entries, 0 to 44999
Data columns (total 62 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   target_default                                   41085 non-null  object 
 1   score_1                                          41085 non-null  object 
 2   score_2                                          41085 non-null  object 
 3   score_3                                          41085 non-null  float64
 4   score_4                                          41085 non-null  float64
 5   score_5                                          41085 non-null  float64
 6   score_6                                          41085 non-null  float64
 7   risk_rate                                        41085 non-null  float64
 8   last_amount_borrowed                             41085 non-null  float64
 9   last_borrowed_in_months     

We have no missing values and are ready to follow-up with our predictions!

# References
1. https://www.risk-officer.com/Credit_Risk.htm
2. https://www.investopedia.com/terms/c/creditrisk.asp
3. https://www.investopedia.com/terms/f/five-c-credit.asp
4. https://www.sciencedirect.com/topics/economics-econometrics-and-finance/interest-rate-risk